In [5]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import numpy as np
#   If you are getting ModuleNotFoundError, uncomment the following line...
import sys
#   ...and then replace 'your_path' with your path to this CMOR492-DWS directory
#       (if you're pulling from the github, the directory is called CMOR492-DWS as of 19/02/2025)
# sys.path.append('your_path/CMOR492-DWS')
# sys.path.append('/Users/danielsuarez/Documents/Academic/Spring2025/SeniorDesign/CMOR492-DWS/')
sys.path.append("C:\\Users\\gabri\\Documents\\CMOR492-DWS")
from network_construction.network import source_treatment, get_Utown


In [6]:
G = get_Utown()
source_nodes, treatment_nodes = source_treatment(G, 40)  # <-- Specify # starting points for treatment node algorithm

In [7]:
### MODEL PARAMETERS

# TODO: What if we just make pipe size continuous/linear

Path = {}  # Set of shortest paths from each source node i to each treatment node j
NLinks = {}  # Number of edges in each path
L = {}  # Length of each path (distance)

for i in source_nodes:
    for j in treatment_nodes:
        path = nx.shortest_path(G, source=i, target=j, weight='length')
        Path[i, j] = path
        NLinks[i, j] = len(path)-1
        L[i, j] = nx.path_weight(G, path, weight='length')
LE = {e: G.edges[e]['length'] for e in G.edges}  # Length of edge e
EL = {v: G.nodes[v]['elevation'] for v in G.nodes}  # Elevation of node v


D = [0.2, 0.25, 0.3, 0.35, 0.40, 0.45]  # Pipe diameters
CP = {0.05: 8.7, 0.06: 9.5, 0.08: 11,
                       0.1: 12.6, 0.15: 43.5, 0.2: 141,
                       0.25: 151, 0.3: 161, 0.35: 180,
                       0.4: 190, 0.45: 200}  # Cost per unit of pipe


SR = {}  # Production at Source node i
CAP = {}  # Capacity at treatment node j

for node in source_nodes:
    G.nodes[node]['production'] = .17
    SR[node] = .17

total_flow = sum(SR.values())

for node in treatment_nodes:
    G.nodes[node]['capacity'] = 1000
    CAP[node] = 1000

Vmin = 0.6 * 60
Vmax = 3 * 60

CE = 25  # Cost of Excavation
CB = 6  # Cost of Bedding
TR = 44000  # Fixed Cost of Treatment Plant
TRFlow = 100  # Variable Cost of Treatment
PICost = 30

PF = {'0.05': 8.7, '0.06': 9.5, '0.08': 11,
                       '0.1': 12.6, '0.15': 43.5, '0.2': 141,
                       '0.25': 151, '0.3': 161, '0.35': 180,
                       '0.4': 190, '0.45': 200}  # Fixed Cost of Piping

CT = 1000000000  # Cost of trucking
M = 1e6

Smin = 0.01
Smax = 0.1
W = 0.5  # Buffer Width


In [8]:
m = gp.Model()

### DECISION VARIABLES (first period)

x = m.addVars(Path.keys(), vtype=GRB.BINARY, name='x')  # Path ij used
y = m.addVars(treatment_nodes, vtype=GRB.BINARY, name='y')  # treatment at node j
z = m.addVars(G.edges, vtype=GRB.BINARY, name='z')  # edge e used

# | Not used anywhere else, at least in this notebook
# v
# Pipe Size Selection
# d_es_names = []
# for e in G.edges:
#     for s in D:
#         d_es_names.append(f"Edge: {e[0]}---{e[1]} <- Size: {s}")

a = m.addVars(G.edges, D, vtype=GRB.BINARY, name='a')  # Pipe size s at edge e

# Node Elevation
el = m.addVars(G.nodes, vtype=GRB.CONTINUOUS, name='el')  # Elevation at node el_v

# Recourse Amount
r = m.addVars(source_nodes, vtype=GRB.CONTINUOUS, lb=0.0, name='r')  # flow handled at trucking at edge e

# Flow in Edge e
Q = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb=0.0, name='Q')  # Flow in Edge e

# Path Flow
p = m.addVars(Path.keys(), vtype=GRB.CONTINUOUS, lb = 0.0, name='p')

m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2026-02-19


In [9]:
### CONSTRAINTS (first period)

# NODE PRODUCTION MINUS RECOURSE
node_prod_rec = m.addConstrs((p[i, j] >= (SR[i] * x[i, j]) - r[i] for i, j in Path.keys()), name='node_prod_rec')

# TREATMENT CAPACITY
treat_cap = m.addConstrs((gp.quicksum(p[i, j] for i in source_nodes) <= CAP[j] * y[j] for j in treatment_nodes), name='treat_cap')

#  NODE ASSIGNMENT
node_assign = m.addConstrs((gp.quicksum(x[i, j] for j in treatment_nodes) == 1 for i in source_nodes), name='node_assign')

# PIPE SIZING
pipe_sizing = m.addConstrs((gp.quicksum(a[*e, s] for s in D) == z[e] for e in G.edges), name='pipe_sizing')  # ALWAYS BE SURE TO UNPACK e

# TODO: Go through this with John
# FLOW DEFINITION
def is_sublist(short_list, long_list):
    for i in range(len(long_list) - len(short_list) + 1):
        if long_list[i:i + len(short_list)] == short_list:
            return True
    return False

flow_def = m.addConstrs((Q[e] == gp.quicksum(p[i, j] for i, j in Path.keys() if is_sublist(list((e[0], e[1])),Path[i,j])) for e in G.edges), name='flow_def')


# MIN/MAX SLOPE
min_slope = m.addConstrs((el[e[0]] - el[e[1]] >= (LE[e] * Smin) - (M * (1 - z[e])) for e in G.edges), name='min_slope')
max_slope = m.addConstrs((el[e[0]] - el[e[1]] <= (LE[e] * Smax) + (M * (1 - z[e])) for e in G.edges), name='max_slope')

# FLOW VELOCITY LIMIT
flow_vel = m.addConstrs((Q[e] <= Vmax * gp.quicksum((np.pi / 8) * (s**2) * (a[*e, s]) for s in D) for e in G.edges), name='flow_vel')

# PIPES UNDERGROUND
underground = m.addConstrs((el[u] <= EL[u] for u in G.nodes), name='underground')
m.update()
# EDGE ACTIVATION

# TODO: Go through this with John 2
# EDGE ACTIVATION
ePath = {}  # Use this for Edge Activation Constraint
for e, p in Path.items():
    ePath[e] = [(p[l - 1], p[l]) for l in range(1, len(p))]

edge_activate = m.addConstrs((gp.quicksum(z[e] for e in ePath[i, j]) >= NLinks[i, j] * x[i, j] for i, j in Path), name='edge_activate')


# ENVELOPES FOR MANNING

T = 11.9879
P = lambda LE, s: LE / (T * (s**(16/3)))
Qmax = lambda s: Vmax * ((np.pi / 8) * (s**2))


alpha = m.addVars(G.edges, D, lb=0, name='alpha')
beta = m.addVars(G.edges, D, lb=0, name='beta')


alpha_2 = m.addConstrs((alpha[*e, s] >= Q[e] + a[*e, s] * Qmax(s) - ( Qmax(s)) for e in G.edges for s in D), name='alpha_2')
alpha_3 = m.addConstrs((alpha[*e, s] <= Qmax(s) * a[*e, s] for e in G.edges for s in D), name='alpha_3')
alpha_4 = m.addConstrs((alpha[*e, s] <= Q[e] for e in G.edges for s in D), name='alpha_4')
alpha_5 = m.addConstrs((alpha[*e, s] <= Qmax(s) for e in G.edges for s in D), name='alpha_5')

beta_2 = m.addConstrs((beta[*e, s] >= (Qmax(s) * Q[e]) + (Qmax(s) * alpha[*e, s]) - (Qmax(s)**2) for e in G.edges for s in D), name='beta_2')
beta_3 = m.addConstrs((beta[*e, s] <= Qmax(s) * alpha[*e, s] for e in G.edges for s in D), name='beta_3')
beta_4 = m.addConstrs((beta[*e, s] <= Qmax(s) * Q[e] for e in G.edges for s in D), name='beta_4')

# manning_2 = m.addConstrs((el[e[1]] - el[e[0]] + gp.quicksum(P(LE[e], s) * beta[*e, s] for s in D) <= 0 for e in G.edges), name='manning_2')

# ADDED THE BIG M THING HERE BUT IDK IF IT COULD BE IMPROVED
manning_2 = m.addConstrs((el[e[1]] - el[e[0]] + gp.quicksum(P(LE[e], s) * beta[*e, s] for s in D) <= (1-z[e])*M for e in G.edges), name='manning_2')
m.update()

In [10]:
# OBJECTIVE EPXR 1: TREATMENT COSTS

# treat_cost = gp.LinExpr()
# for j in treatment_nodes:
#     treat_cost.addTerms(TR, y[j])
#     for i in source_nodes:
#         treat_cost.addTerms(TRFlow * SR[i], x[i, j])
#
# # OBJECTIVE EXPR 2: EXCAVATION COSTS
# excav_cost_f = lambda u, v: gp.QuadExpr(CE * (((EL[u] - el[u]) + (EL[v] - el[v])) / 2) * LE[u, v] * gp.quicksum(s + ((2*W) * d[u, v, s]) for s in D))
#
# # OBJECTIVE EXPR 3: BEDDING COSTS
# bed_cost_f = lambda u, v: gp.LinExpr(CB * LE[u, v] * gp.quicksum(s + ((2*W) * d[u, v, s]) for s in D))
# # OBJECTIVE EXPR 4: PIPE COSTS
# pipe_cost_f = lambda u, v: gp.LinExpr(LE[u, v] * gp.quicksum(CP[s] * d[u, v, s] for s in D))
#
# excav_bed_cost = gp.quicksum(excav_cost_f(u, v) + bed_cost_f(u, v) + pipe_cost_f(u, v) for u, v in G.edges)
#
# # OBJECTIVE EXPR 5: RECOURSE TRUCKING
#
# rec_cost = gp.LinExpr()
# for i in source_nodes:
#     rec_cost.addTerms(CT, r[i])
#
# m.setObjective(treat_cost + excav_bed_cost + rec_cost, GRB.MINIMIZE)

m.setObjective(0, GRB.MINIMIZE)

m.update()
print(f"Model has {m.NumVars} variables and {m.NumConstrs} constraints.")

Model has 24786 variables and 38926 constraints.


In [11]:
# m.write("singleperiod_nocontext2.lp")
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 9 5900HX with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 38926 rows, 24786 columns and 286316 nonzeros
Model fingerprint: 0x11badabe
Variable types: 14060 continuous, 10726 integer (10726 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+06]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 27525 rows and 12307 columns
Presolve time: 0.65s
Presolved: 11401 rows, 12479 columns, 103321 nonzeros
Variable types: 9405 continuous, 3074 integer (3074 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.71 seconds (1.07 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 

In [12]:
### Multiperiod-related variables
n_periods = 3
periods = list(range(1, n_periods + 1)) # `periods` can be an arbitrary list/tuple, here we just make it [1,2,3]
(xt, yt, zt, at, elt, rt, Qt, pt) = [{period: None for period in periods} for _ in (x,y,z,a,el,r,Q,p)] # shorthand for creating dictionaries (wrap this in function?)

In [13]:
# Write the value of each decision variable in the first period into the historical dictionaries
for var, history_dict in zip((x,y,z,a,el,r,Q,p), (xt,yt,zt,at,elt,rt,Qt,pt)):
    history_dict[periods[0]] = var.copy() # Make sure to copy the values, so that changing `x` for example doesn't change the history

In [ ]:
### ADD MULTIPERIOD DECISION VARIABLES AND CONSTRAINTS

current_period_index = 1

## Variables 
# What new size pipe needs to be installed, if any
d = m.addVars(G.edges, D, vtype=GRB.BINARY, name='d')

# Whether elevation at vertex v changes
c = m.addVars(G.nodes, vtype=GRB.BINARY, name='c')

## Constraints
# CHANGE IN PIPE SIZE (ensures that at most 1 new pipe size can be selected)
pipe_size_change = m.addConstrs((gp.quicksum(d[*e, s] for s in D) <= 1 for e in G.edges), name='pipe_size_change')  # Always unpack e

# a-CONSTRAINT (ensures that a truly represents the accurate pipe size)
a_constraint = m.addConstrs((a[*e, s] <= (d[*e, s] + at[periods[current_period_index-1]][*e, s]) for s in D for e in G.edges), name='a_constraint')



In [3]:
tuple1 = (1, 2)
print(tuple1[0:])


(1, 2)


In [ ]:
### ADD VARIABLES AND CONSTRAINTS FOR PERIOD 2

d = m.addVars(G.edges, D, vtype=GRB.BINARY, name='d')  # NEW pipe size s at edge e, if the pipe size has changed




In [21]:
for v in r:
    if r[v].X > 0:
        print(r[v].VarName, r[v].X)

r[59079984] 1.0
r[59080764] 1.0
r[59081555] 1.0
r[59081615] 1.0
r[59081625] 1.0
r[59081682] 1.0
r[59081790] 1.0
r[59081845] 1.0
r[59082151] 1.0
r[59082376] 1.0
r[59082984] 1.0
r[59084971] 1.0
r[59084975] 1.0
r[59084990] 1.0
r[59091067] 1.0
r[59091076] 1.0
r[59095757] 1.0
r[59095758] 1.0
r[59096570] 1.0
r[59097709] 1.0
r[59101042] 1.0
r[59101390] 1.0
r[59102119] 1.0
r[59104191] 1.0
r[59105503] 1.0
r[59105510] 1.0
r[59106124] 1.0
r[59106145] 1.0
r[59108226] 1.0
r[59108232] 1.0
r[59108749] 1.0
r[59110218] 1.0
r[59110295] 1.0
r[59110368] 1.0
r[59112331] 1.0
r[59112337] 1.0
r[59112870] 1.0
r[59113590] 1.0
r[59115071] 1.0
r[59115076] 1.0
r[59115377] 1.0
r[59115384] 1.0
r[59116275] 1.0
r[59116279] 1.0
r[59116284] 1.0
r[59116298] 1.0
r[59117099] 1.0
r[59117103] 1.0
r[59117109] 1.0
r[59120062] 1.0
r[59120066] 1.0
r[59120113] 1.0
r[59123240] 1.0
r[59123827] 1.0
r[59123873] 1.0
r[59123898] 1.0
r[59123905] 1.0
r[59123911] 1.0
r[59123919] 1.0
r[59123924] 1.0
r[59123928] 1.0
r[59123932] 1.0
r[591239

In [9]:
m.computeIIS()

for c in m.getConstrs():
    if c.IISConstr:
        print(f"Constraint {c.ConstrName} is in the IIS")

for v in m.getVars():
    if v.IISLB > 0:
        print(f"Lower bound of {v.VarName} is in the IIS")
    elif v.IISUB > 0:
        print(f"Upper bound of {v.VarName} is in the IIS")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 9 5900HX with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

IIS computation: initial model status unknown, solving to determine model status
Presolve removed 27525 rows and 12506 columns
Presolve time: 0.66s
Presolved: 11401 rows, 12479 columns, 103321 nonzeros
Variable types: 9405 continuous, 3074 integer (3074 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.73 seconds (1.07 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
IIS runtime: 0.74 seconds (1.07 work units)


GurobiError: Cannot compute IIS on a feasible model

In [22]:
for v in m.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

x[59079984,59129525] 1.0
x[59080764,4230838774] 1.0
x[59080765,4230838774] 1.0
x[59080775,59107322] 1.0
x[59081555,59081566] 1.0
x[59081564,59081566] 1.0
x[59081615,59081605] 1.0
x[59081625,59081605] 1.0
x[59081682,4230838774] 1.0
x[59081782,59146293] 1.0
x[59081790,59146293] 1.0
x[59081840,59097341] 1.0
x[59081845,59097341] 1.0
x[59082151,59153773] 1.0
x[59082376,4231830135] 1.0
x[59082984,4230838774] 1.0
x[59084971,4230838774] 1.0
x[59084975,4230838774] 1.0
x[59084990,4230838774] 1.0
x[59091067,59091084] 1.0
x[59091076,59091084] 1.0
x[59094292,59118652] 1.0
x[59095026,59080788] 1.0
x[59095757,59097341] 1.0
x[59095758,59097341] 1.0
x[59096556,59123881] 1.0
x[59096570,59123881] 1.0
x[59096600,59096602] 1.0
x[59097709,59097341] 1.0
x[59101042,59091084] 1.0
x[59101386,59101385] 1.0
x[59101390,59101385] 1.0
x[59102119,59107322] 1.0
x[59103180,59103172] 1.0
x[59104191,59079200] 1.0
x[59105503,59101385] 1.0
x[59105510,59101385] 1.0
x[59106124,59115372] 1.0
x[59106145,59115372] 1.0
x[5910822

In [ ]:
# Call after past value dictionaries have been initialized and the model has been run once
record_period(periods[0], (xt,y,z,el,r,Q,p), ((xt,yt,zt,elt,rt,Qt,pt))) # Must match order


In [84]:
periods = [1,2,3]
xt = {period: None for period in periods}
xt[periods[0]] = x.copy()
print(f"{type(x) = }")
print(f"{type(xt[periods[0]]) = }")
print(f"{list(Path.keys())[0] = }")
print(f"{xt[periods[0]][list(Path.keys())[0]] = }")

type(x) = <class 'gurobipy._core.tupledict'>
type(xt[periods[0]]) = <class 'dict'>
list(Path.keys())[0] = (59079984, 59128065)
xt[periods[0]][list(Path.keys())[0]] = <gurobi.Var x[59079984,59128065] (value 0.0)>


In [ ]:
path_key_list = [list(key) for key in Path.keys()]
print(f"{path_key_list[0] = }")
print(f"x{list(path_key_list[0])}")

x_vals = [var.X for (index, var) in enumerate(m.getVars()) if ((index < len(path_key_list)) and (f"x{path_key_list[index][0]}" in var.VarName))]
var0 = m.getVars()[0]
print(f"{var0 = }")
print(f"{'x' in var0.VarName = }, {f'{path_key_list[0][0]}' in var0.VarName = }, {f'{path_key_list[0][1]}' in var0.VarName = }")
print(f"{path_key_list[0] = }")
print(f"{len(x_vals) = }")

path_key_list[0] = [59079984, 59128065]
x[59079984, 59128065]
var0 = <gurobi.Var x[59079984,59128065] (value 0.0)>
'x' in var0.VarName = True, f'{path_key_list[0][0]}' in var0.VarName = True, f'{path_key_list[0][1]}' in var0.VarName = True
path_key_list[0] = [59079984, 59128065]
len(x_vals) = 0


In [ ]:
x_0 = {str(xv) : x[xv].X for xv in x}
y_0 = {yv : y[yv].X for yv in y}
z_0 = {str(zv) : z[zv].X for zv in z}

d_0 = {str(dv) : a[dv].X for dv in a}
el_0 = {elv: el[elv].X for elv in el}

In [ ]:
import json

In [ ]:
with open("x_sol.json", "w") as f:
    json.dump(x_0, f)

In [ ]:
with open("z_sol.json", "w") as f:
    json.dump(z_0, f)

with open("d_sol.json", "w") as f:
    json.dump(d_0, f)

with open("el_sol.json", "w") as f:
    json.dump(el_0, f)